# **Projeto de Exploração e análise de dados de crédito com SQL**

![](https://github.com/layssasantos/Projeto-EDA-Credito-SQL/blob/main/IMAGENS_PROJETO/What_is_SQL_Database.png?raw=true)

---

## Tópicos
1. Introdução
2. Sobre os dados
3. Criação da Tabela
4. Exploração de Dados
5. Análise de Dados
6. Conclusão

---
 
## 1. Introdução

Este projeto consiste em realizar uma Análise Exploratória de Dados (EDA) de um conjunto de dados de crédito de clientes de um banco através da linguagem SQL, tendo sido utilizado o AWS Athena para a execução da querys.


## 2. Sobre os dados 

Os dados representam as informações de clientes de um banco e contam com as seguintes colunas: 

* idade = idade do cliente
* sexo = sexo do cliente (F ou M)
* dependentes = número de dependentes do cliente
* escolaridade = nível de escolaridade do clientes
* salario_anual = faixa salarial do cliente
* tipo_cartao = tipo de cartao do cliente
* qtd_produtos = quantidade de produtos comprados nos últimos 12 meses
* iteracoes_12m = quantidade de iterações/transacoes nos ultimos 12 meses
* meses_inativo_12m = quantidade de meses que o cliente ficou inativo
* limite_credito = limite de credito do cliente
* valor_transacoes_12m = valor das transações dos ultimos 12 meses
* qtd_transacoes_12m  = quantidade de transacoes dos ultimos 12 meses

Ressalte-se que neste projeto foi utilizada apenas uma parte dos dados do arquivo CSV disponibilizado neste [link](https://github.com/andre-marcos-perez/ebac-course-utils/tree/main/dataset).


## 3. Criação da tabela

Para iniciar o projeto, foi criado um novo novo bucket s3 (bucket-transacoes-projeto-layssa), para fins de depósito do arquivo csv. Em seguida, na AWS Athena, foi criada a tabela 'credito' a partir deste arquivo csv.

**Query:**
CREATE EXTERNAL TABLE IF NOT EXISTS default.credito ( 

  `idade` int,
  
  `sexo` string,
  
  `dependentes` int,
  
  `escolaridade` string,
  
  `estado_civil` string,
  
  `salario_anual` string,
  
  `tipo_cartao` string, 
  
  `qtd_produtos` bigint,
  
  `iteracoes_12m` int,
  
  `meses_inativo_12m` int,
  
  `limite_credito` float,
  
  `valor_transacoes_12m` float,
  
  `qtd_transacoes_12m` int 
  
)

ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'

WITH SERDEPROPERTIES (

  'serialization.format' = ',',
  
  'field.delim' = ','
  
) LOCATION 's3://bucket-transacoes-projeto-layssa/'


TBLPROPERTIES ('has_encrypted_data'='false');

## 4. Exploração de dados

Nesta etapa, vamos entender melhor as informações contidas em nossa tabela. Assim, é preciso explorar os dados.

* **Contagem dos dados**

**Query**: SELECT count(*) FROM credito;
> Resposta: 2564 linhas

* **Visualização dos dados**

Visualização das 10 primeiras linhas

**Query**: SELECT * FROM credito LIMIT 10;
![Dez primeiras linhas do dataset](https://github.com/layssasantos/Projeto-EDA-Credito-SQL/blob/main/IMAGENS_PROJETO/visualiza%C3%A7%C3%A3o%20da%20tabela.JPG?raw=true)

* **Descrição dos tipos de dados presente em cada coluna da tabela**

**Query:** DESCRIBE credito;

![Descrição dos tipos de dados](https://github.com/layssasantos/Projeto-EDA-Credito-SQL/blob/main/IMAGENS_PROJETO/ED_descri%C3%A7%C3%A3o.JPG?raw=true)

* **Valores nulos**

Na exploração dos dados é sempre relevante verificarmos se existem valores nulos, para que possamos realizar uma manipulação assertiva dos dados. 

Deste modo, vamos olhar mais atentamente para as colunas que não são numéricas.

**| Escolaridade**

**Query:** SELECT DISTINCT escolaridade FROM credito;

![Tipos de escolaridade](https://github.com/layssasantos/Projeto-EDA-Credito-SQL/blob/main/IMAGENS_PROJETO/ED_escolaridade.jpg?raw=true)


**| Estado civil**

**Query:** SELECT DISTINCT estado_civil  FROM credito;

![Tipos de estado civil](https://github.com/layssasantos/Projeto-EDA-Credito-SQL/blob/main/IMAGENS_PROJETO/ED_estado%20civil.jpg?raw=true)

**| Salário anual**

**Query:** SELECT DISTINCT salario_anual  FROM credito;

![Tipos de salario anual](https://github.com/layssasantos/Projeto-EDA-Credito-SQL/blob/main/IMAGENS_PROJETO/ED_sal%C3%A1rio.jpg?raw=true)

**| Cartão**

**Query:** SELECT DISTINCT tipo_cartao  FROM credito;

![Tipos de cartão](https://github.com/layssasantos/Projeto-EDA-Credito-SQL/blob/main/IMAGENS_PROJETO/ED_cart%C3%A3o.JPG?raw=true)

Diante do exposto, com exceção dos tipos de cartão, é possível verificar valores nulos nas demais colunas exploradas, o que pode indicar um erro na hora de processar a informação em um cadastro, ou dados faltantes. Isso nos demandará maior atenção no momento da análise dos dados, em termos de filtragem.

## **5. Análise de dados**

Uma vez concluída a exploração dos dados, vamos dar início à análise das informações para buscar entender o que está acontecendo no banco de dados. 

Nesta etapa, podemos fazer algumas perguntas de negócio para obter insights:

* **Quanto às faixas salariais, quantos clientes pertencem à cada uma delas?**

**Query:** SELECT COUNT(*), salario_anual FROM credito GROUP BY salario_anual;

![Quantidade para cada faixa salarial](https://github.com/layssasantos/Projeto-EDA-Credito-SQL/blob/main/IMAGENS_PROJETO/AD_clientes%20por%20sal%C3%A1rio.JPG?raw=true)

> É possível observar que o maior grupo de pessoas possuem salário menor que 40K e que uma minoria possui salário na faixa de 120K+. Além disso, 235 clientes não informaram seu salário ou não consta a faixa salarial. 
Com o gráfico de barras gerado a partir do arquivo csv extraído dos dados conseguimos visualizar melhor:
![Gráfico_1](https://github.com/layssasantos/Projeto-EDA-Credito-SQL/blob/main/IMAGENS_PROJETO/GR%C3%81FICOS/Gr%C3%A1fico_AD_sal%C3%A1rio.jpg?raw=true)

* **Quais faixas salariais predominam em cada tipo de cartão?**

**Query:** SELECT COUNT(*) AS quantidade, salario_anual, tipo_cartao
FROM credito
WHERE salario_anual != 'na'
GROUP BY salario_anual, tipo_cartao
ORDER BY quantidade desc;

![Tipo de cartão por faixa salarial](https://github.com/layssasantos/Projeto-EDA-Credito-SQL/blob/main/IMAGENS_PROJETO/AD_cart%C3%A3o%20por%20sal%C3%A1rio.jpg?raw=true)

> Denota-se que o cartão Blue concentra o maior número de clientes de todas as faixas salariais, predominando aqueles que tem salário anual menor que 40K. Por outro lado, os cartões Silver e Platinum demonstram ter em sua maioria clientes com salários mais altos entre 80K e 120K. Os cartões Gold e Platinum possuem uma concentração menor de clientes de todas as faixas salariais, pelo menos é o que se pode afirmar nesta fração menor dos dados originais. Vejamos a projeção no gráfico:
![Gráfico_2](https://github.com/layssasantos/Projeto-EDA-Credito-SQL/blob/main/IMAGENS_PROJETO/GR%C3%81FICOS/Gr%C3%A1fico_AD_cart%C3%A3o%20por%20sal%C3%A1rio.jpg?raw=true)

* **Como se dá a distribuição de homens e mulheres na tabela?**

**Query:** SELECT COUNT(*), sexo FROM credito GROUP BY sexo;

![Quantidade para cada sexo](https://github.com/layssasantos/Projeto-EDA-Credito-SQL/blob/main/IMAGENS_PROJETO/AD_sexo.JPG?raw=true)

> Torna-se evidente que a maioria dos clientes é dominada pelo sexo masculino. 
Com o csv extraído dos dados foi gerado um gráfico em pizza para melhor visualização da proporção entre masculino/feminino:
![Quantidade para cada sexo - gráfico](https://github.com/layssasantos/Projeto-EDA-Credito-SQL/blob/main/IMAGENS_PROJETO/GR%C3%81FICOS/qtde_por_sexo.png?raw=true)

* **Visando uma estratégia de marketing mais assertiva, qual é a idade dos clientes?**

**Query:** SELECT AVG(idade) AS media_idade, MIN(idade) AS min_idade, MAX(idade) AS max_idade, sexo FROM credito GROUP BY sexo;

![Média de idades por sexo](https://github.com/layssasantos/Projeto-EDA-Credito-SQL/blob/main/IMAGENS_PROJETO/AD_idade.JPG?raw=true)

> Se observa que a idade média dos dois sexos é muito similar,bem como a menor idade dos dois sexos são iguais. Apesar da idade máxima dos homens ser maior, não é algo que possui tanta relevância em razão da diferença não ser gritante.


* **Quais são as características dos clientes menos ativos?**

**Query:** SELECT AVG(meses_inativo_12m) AS media_inatividade, estado_civil, salario_anual, sexo 
FROM credito 
WHERE estado_civil != 'na' AND salario_anual != 'na'
GROUP BY estado_civil, salario_anual, sexo 
ORDER BY media_inatividade desc
LIMIT 10;

![Clientes com maior inatividade](https://github.com/layssasantos/Projeto-EDA-Credito-SQL/blob/main/IMAGENS_PROJETO/AD_inatividade.JPG?raw=true)

> Fica bastante claro que, com base nas características selecionadas, os homens lideram a inatividade, principalmente os casados, independente da renda. Além disso, divorciados, incluindo o sexo feminino, demonstram uma média de meses inativos alta, aparecendo com frequência no topo da tabela.

* **O estado civil influencia nos valores de transações e no limite de crédito?**

**Query:** SELECT AVG(valor_transacoes_12m) AS media_valor_transacoes, AVG(limite_credito) AS media_limite, estado_civil, sexo FROM credito WHERE estado_civil != 'na' GROUP BY estado_civil, sexo ORDER BY media_limite DESC;

![Limite de crédito e transações por estado civil](https://github.com/layssasantos/Projeto-EDA-Credito-SQL/blob/main/IMAGENS_PROJETO/AD_estadocivil.JPG?raw=true)

>É possível observar que os divorciados possuem maiores limite de crédito, bem como gastam mais, similar aos solteiros. Já os clientes casados possuem menores limites de crédito, tanto homens, quanto mulheres, e gastam menos em transações.

* **E a faixa salarial, ela tem alguma influência no limite?**

**Query:** SELECT AVG(qtd_produtos) AS qts_produtos, AVG(valor_transacoes_12m) AS media_valor_transacoes, AVG(limite_credito) AS media_limite, sexo, salario_anual
FROM credito
WHERE salario_anual != 'na'
GROUP BY sexo, salario_anual 
ORDER BY AVG(valor_transacoes_12m) DESC;

![Valor salario_anualLimite](https://github.com/layssasantos/Projeto-EDA-Credito-SQL/blob/main/IMAGENS_PROJETO/AD_limite%20por%20sal%C3%A1rio.JPG?raw=true) 

>É evidente que sim, as pessoas que tem menor faixa salarial também apresentam menor limite de credito!

* **Quais são as características dos clientes que possuem os maiores creditos?**

**Query:** 
SELECT MAX(limite_credito) AS limite_credito, escolaridade, tipo_cartao, sexo 
FROM credito
WHERE escolaridade != 'na' AND tipo_cartao != 'na' 
GROUP BY  escolaridade, tipo_cartao, sexo 
ORDER BY limite_credito DESC
LIMIT 10;

![Valor limite](https://github.com/layssasantos/Projeto-EDA-Credito-SQL/blob/main/IMAGENS_PROJETO/AD_maiores%20limites.JPG?raw=true)

> Os homens possuem os maiores limites de crédito. Além disso, a escolaridade não parece influenciar tanto no crédito, tendo em vista que o limite mais alto é oferecido para um homem sem educação formal. Os cartões gold, silver, platinum e blue estão bem divididos entre os clientes com os maiores limites. Por fim, o cartão também parece não estar relacionado com a escolaridade nem com o limite. 


* **Apesar de não terem os maiores limites, as mulheres gastam mais ou menos que os homens?**

**Query:** 
SELECT MAX(valor_transacoes_12m) AS maior_valor_gasto, AVG(valor_transacoes_12m) AS  media_valor_gasto, MIN(valor_transacoes_12m) AS min_valor_gasto, sexo 
FROM credito
GROUP BY sexo;


![Valor transacoes/sexo](https://github.com/layssasantos/Projeto-EDA-Credito-SQL/blob/main/IMAGENS_PROJETO/AD_gastos%20por%20sexo.JPG?raw=true)

> Apesar da diferença nos limites, os gastos de homens e mulheres são similares!

## 6. Conclusão

A partir da análise extraídas do dataset de crédito, é possível extrair alguns insights interessantes:  

- A maior parte dos clientes possuem salário anual menor que 40 mil;
- O cartão Blue tem o maior número de clientes por faixa salarial, em especial aqueles que ganham menos de $40 mil por ano;
- A maior parte dos clientes da base de dados é formada por homens;
- Os clientes do sexo masculino são os que apresentam maior inatividade;
- Os clientes casados possuem menores limites de crédito e transações, independentemente do sexo;
- A faixa salarial impacta diretamente no limite de crédito;
- A escolaridade não parece influenciar no limite nem no tipo do cartão;
- Dentre os menores limites não há presença de cartão platinum;
- Os clientes com maiores limites são em sua maioria homens, apesar dos gastos entre os dois sexos serem similares. Uma exploração maior dos dados pode explicar a disparidade dos limites de crédito ou pode ser um problema cultural a ser repensado;